In [1]:
require 'torch'
require 'nn'
require 'image'
require 'optim'

In [2]:
loadcaffe_wrap = require 'loadcaffe_wrapper'
-- cjson = require 'cjson'
json = require 'json'
string = require 'string'
m = require 'manifold'

In [3]:

cmd = torch.CmdLine()

-- Hacking torch
cmd:option('-start_at', 1, 'index to start at – worst hack I\'ve ever written')
cmd:option('-iter', 100, 'how many images to run over – please don\'t segfault')
cmd:option('-img_size', 512, 'all images will be resized to this max dimension' )
cmd:option('-name', '', 'name to attach to output')
cmd:option('-thumb_size', 100, 'thumbnail size')


-- Basic options
cmd:option('-style_dir', 'data/picasso_cubism/', 'Style input directory')
cmd:option('-tmp_dir', 'tmp/', 'Directory to store vectors on disk')
cmd:option('-gpu', -1, 'Zero-indexed ID of the GPU to use; for CPU mode set -gpu = -1')

-- Other options
cmd:option('-pooling', 'max', 'max|avg')
cmd:option('-proto_file', 'models/VGG_ILSVRC_19_layers_deploy.prototxt')
cmd:option('-model_file', 'models/VGG_ILSVRC_19_layers.caffemodel')

cmd:option('-content_layers', 'relu4_2', 'layers for content')
cmd:option('-style_layers', 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1', 'layers for style') -- tbh all but relu6 and relu7, which cause size mismatches
                            -- 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1'
                            -- 'relu1_1,relu1_2,relu2_1,relu2_2,relu3_1,relu3_2,relu3_3,relu3_4,relu4_1,relu4_2,relu4_3,relu4_4,relu5_1,relu5_2,relu5_3,relu5_4'


In [5]:
function cached(label) -- check if a cached version of vec exists
    local filename = params.tmp_dir .. 'cache/' .. label .. '.cache'
    local f = io.open(filename,"r")
    if f ~= nil then
        io.close(f)
        return true
    else
        return false
    end
end

function load_cache(label)
    print("loading  from cache...")
    local filename = params.tmp_dir .. 'cache/' .. label .. '.cache'
    local f = torch.load(filename)
    return f
end

function cache(file, label)
    if paths.dir(params.tmp_dir .. 'cache/') == nil then paths.mkdir(params.tmp_dir .. 'cache/') end

    local filename = params.tmp_dir .. 'cache/' .. label .. '.cache'
    local f = torch.save(filename, file)
    print(string.format("cached  %s", label))
    return true
end

In [4]:

-------------------------------------------------------------------------------------


-- Returns a network that computes the CxC Gram matrix from inputs
-- of size C x H x W – jcjohnson's version
function GramMatrix()
    local net = nn.Sequential()
    net:add(nn.View(-1):setNumInputDims(2))
    local concat = nn.ConcatTable()
    concat:add(nn.Identity())
    concat:add(nn.Identity())
    net:add(concat)
    net:add(nn.MM(false, true))
    return net
end


-- utility function to reshape a tensor from M x N x ... to an MxN array
function flatten(t)
    return torch.view(t, -1)
end

-- a function to do memory optimizations by 
-- setting up double-buffering across the network.
-- this drastically reduces the memory needed to generate samples.
-- from soumith/dcgan.torch
function optimizeInferenceMemory(net)
    local finput, output, outputB
    net:apply(
        function(m)
            if torch.type(m):find('Convolution') then
                finput = finput or m.finput
                m.finput = finput
                output = output or m.output
                m.output = output
            elseif torch.type(m):find('ReLU') then
                m.inplace = true
            elseif torch.type(m):find('BatchNormalization') then
                outputB = outputB or m.output
                m.output = outputB
            end
    end)
end


function Style2Vec(cnn, gram, img)
    --[[ runs img through cnn, saving the output tensor at each of style_layers

    -- FOR NOW, only returns relu4_1

    relu1_1 : FloatTensor - size: 64x64
    relu1_2 : FloatTensor - size: 64x64
    relu2_1 : FloatTensor - size: 128x128
    relu2_2 : FloatTensor - size: 128x128
    relu3_1 : FloatTensor - size: 256x256
    relu3_2 : FloatTensor - size: 256x256
    relu3_3 : FloatTensor - size: 256x256
    relu3_4 : FloatTensor - size: 256x256
    relu4_1 : FloatTensor - size: 512x512
    relu4_2 : FloatTensor - size: 512x512
    relu4_3 : FloatTensor - size: 512x512
    relu4_4 : FloatTensor - size: 512x512
    relu5_1 : FloatTensor - size: 512x512
    relu5_2 : FloatTensor - size: 512x512
    relu5_3 : FloatTensor - size: 512x512
    relu5_4 : FloatTensor - size: 512x512
    
    Returns a Lua table with the above key-value pairs. 

    
    --]]
    
    local next_style_idx = 1
    local net = nn.Sequential()
    local style_layers = params.style_layers:split(',')
    
    local style_vector = nil 

    -- THIS GUY THIS GUY THIS GUY THIS GUY
    -- nn.JoinTable(1):forward{x, y, x}:float()

    -- Build up net from cnn
    
    for i = 1, #cnn do

        if next_style_idx <= #style_layers then
            local layer = cnn:get(i)
            local layer_name = layer.name

            if params.gpu >= 0 then layer = layer:cuda() end

            net:add(layer)
            
            -- now to grab style layers
            
            if (layer_name == style_layers[next_style_idx]) then
                local target_features = net:forward(img)
                local target_i = gram:forward(target_features)
                target_i:div(target_features:nElement())
                
                -- add the current gram matrix (flattened) to style_vector
                local curr = flatten(target_i):float()
                if style_vector == nil then
                    style_vector = curr
                else
                    style_vector = nn.JoinTable(1):forward({style_vector, curr}):float()
                end

                next_style_idx = next_style_idx + 1     
            end
        end
    end

    collectgarbage(); collectgarbage()
    return style_vector
end


function save_json(filename, file)  
    local filename = params.tmp_dir .. filename .. '.json'
    local json_string = json.encode(file)
    local f = assert(io.open(filename, 'w'))

    f:write(json_string)

    return true
end


function load(label) -- load and preprocess image

    -- Preprocess an image before passing it to a Caffe model.
    -- We need to rescale from [0, 1] to [0, 255], convert from RGB to BGR,
    -- and subtract the mean pixel. [jcjohnson]
    function preprocess(img)
      local mean_pixel = torch.DoubleTensor({103.939, 116.779, 123.68})
      local perm = torch.LongTensor{3, 2, 1}
      local img = img:index(1, perm):mul(256.0)
      mean_pixel = mean_pixel:view(3, 1, 1):expandAs(img)
      img:add(-1, mean_pixel)
      if params.gpu >= 0 then img = img:cuda() end
      return img
    end

    -- load our image
    local ok, img = pcall(image.load, params.style_dir .. label .. '.jpg')
    if not ok then 
        print('error loading image')
        return nil
    end

    if img:size()[1] ~= 3 then
        print('Not enough dimensions on this one')
        return nil
    end

    -- save thumbnail
    assert(save_thumb(img, label))

    -- preprocess for return
    img = image.scale(img, params.img_size, 'bilinear')
    img = preprocess(img):float()

    return img
end


function save_thumb(img, label)
    local thumbs = params.tmp_dir .. 'thumbs/'
    if paths.dir(thumbs) == nil then paths.mkdir(thumbs) end

    local thumb = image.scale(img, params.thumb_size, 'bilinear')
    image.save(thumbs .. label .. '.jpg', thumb)
    return true
end

In [8]:
arg = {}
params = cmd:parse(arg)
if paths.dir(params.tmp_dir) == nil then paths.mkdir(params.tmp_dir) end

In [9]:
-- gpu

if params.gpu >= 0 then
    require 'cutorch'
    require 'cunn'
    cutorch.setDevice(params.gpu + 1)
else
    params.backend = 'nn-cpu'
end

In [10]:
-- get sorted
sorted = {}

for f in paths.iterfiles(params.style_dir) do    
    if string.match(f, '.jpg') then
        label = string.split(f, '.jpg')[1]
        table.insert(sorted, label)
    end
end

table.sort(sorted)
for i,n in ipairs(sorted) do print(i, n) end


1	ARTSTOR_103_41822001426012	
2	BRYN_MAWR_95511124810	
3	BRYN_MAWR_95511125220	
4	BRYN_MAWR_95511125221	
5	BRYN_MAWR_95511128363	
6	BRYN_MAWR_9559344931	
7	BRYN_MAWR_9559344942	
8	BRYN_MAWR_9559344968	
9	BRYN_MAWR_9559345042	
10	BRYN_MAWR_9559345159	
11	BRYN_MAWR_9559352713	
12	LESSING_ART_10310751494	


In [11]:
-- load caffe network image

local cnn = loadcaffe_wrap.load(params.proto_file, params.model_file, params.backend):float()
local gram = GramMatrix():float()
if params.gpu >= 0 then 
    cnn = cnn:cuda()
    gram = gram:cuda() 
end
optimizeInferenceMemory(cnn)

collectgarbage(); collectgarbage()

Successfully loaded models/VGG_ILSVRC_19_layers.caffemodel


conv1_1: 64 3 3 3


conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3
conv3_1: 256 128 3 3
conv3_2: 256 256 3 3
conv3_3: 256 256 3 3


conv3_4: 256 256 3 3


conv4_1: 512 256 3 3


...

conv5_3: 512 512 3 3


conv5_4: 512 512 3 3


fc6: 1 1 25088 4096


fc7: 1 1 4096 4096


fc8: 1 1 4096 1000


In [12]:

-- Run Style2Vec on image by image

ct = 1
ct2 = 1
i = params.start_at

vecs = nil
imgs = nil
out = {}

while (i < #sorted) do
    label = sorted[i]
    io.write(ct .. ' ' .. label .. ':\t')        --      .. params.style_layers .. ' ...' 

    local timer = torch.Timer()
    local vec = nil

    local img = load(label)

    if cached(label) then
        vec = load_cache(label)
    else        
        if img == nil then
            print('error loading image')        -- this error doesn't check if cache exists
        else
            vec = Style2Vec(cnn, gram, img)
            cache(vec, label)
        end
    end

    if vec ~= nil then
        if vecs == nil then
            vecs = vec
        else
            vecs = nn.JoinTable(1):forward({vecs, vec}):float()
            ct = ct + 1
        end

        -- resize all images to 512x512 before flattening to preserve dims
        local std_img = image.scale(img, 512, 512, 'bicubic')
        local flat_std = flatten(std_img)

        if imgs == nil then
            imgs = flat_std
        else
            imgs = nn.JoinTable(1):forward({imgs, flat_std}):float()
            ct2 = ct2 + 1
        end
    
        out[ct] = label
    end

    i = i + 1
    if ct > params.iter then break end
    collectgarbage(); collectgarbage()

    print(string.format("elapsed time: %.2f\n", timer:time().real))
end

1 ARTSTOR_103_41822001426012:	loading  from cache...	
elapsed time: 0.31
	
1 BRYN_MAWR_95511124810:	loading  from cache...	
elapsed time: 0.18
	
2 BRYN_MAWR_95511125220:	loading  from cache...	
elapsed time: 0.17
	
3 BRYN_MAWR_95511125221:	loading  from cache...	
elapsed time: 0.19
	
4 BRYN_MAWR_95511128363:	loading  from cache...	
elapsed time: 0.23
	
5 BRYN_MAWR_9559344931:	loading  from cache...	
elapsed time: 0.14
	
6 BRYN_MAWR_9559344942:	loading  from cache...	
elapsed time: 0.14
	
7 BRYN_MAWR_9559344968:	loading  from cache...	


elapsed time: 0.26
	
8 BRYN_MAWR_9559345042:	

loading  from cache...	


elapsed time: 0.24
	
9 BRYN_MAWR_9559345159:	

loading  from cache...	


elapsed time: 0.18
	
10 BRYN_MAWR_9559352713:	

loading  from cache...	


elapsed time: 0.26
	


In [13]:
-- clean up a little
cnn = nil
style_images = nil
collectgarbage(); collectgarbage()

In [14]:
-- reshape into rows for export and t-SNE
print('reshaping vecs: ')
vecs = vecs:view(ct, -1)
imgs = imgs:view(ct2, -1)

reshaping vecs: 	


In [15]:
print(#vecs)
print(#imgs)


     11
 610304
[torch.LongStorage of size 2]


     11
 786432
[torch.LongStorage of size 2]



In [16]:
local i = 8
local embedding = tsne(vecs, i)
assert(save_json(params.name .. i .. 'embedding', embedding:totable()))

[string "function tsne(vecs, perplexity)..."]:3: attempt to call field 'run_bhtsne' (a nil value)
stack traceback:
	[string "function tsne(vecs, perplexity)..."]:3: in function 'tsne'
	[string "local i = 8..."]:2: in main chunk
	[C]: in function 'xpcall'
	/Users/razi/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/razi/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/razi/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/razi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/razi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/razi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/razi/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	[string "arg={'/Users/razi/.ipython/profile_default/se..."]:1: in main chunk: 

In [24]:
tsne(vecs, 3)

Computing P-values...	


Running t-SNE...	


Iteration 10: KL divergence is 18.963903034566	


Iteration 20: KL divergence is 19.603884907789	


Iteration 30: KL divergence is 16.194569585329	


...

Iteration 950: KL divergence is 0.13896321393287	


Iteration 960: KL divergence is 0.13867815760612	
Iteration 970: KL divergence is 0.13840819229855	


Iteration 980: KL divergence is 0.13815211300714	


Iteration 990: KL divergence is 0.13790685719931	


Iteration 1000: KL divergence is 0.13767317801906	
 124.9486 -191.9213
 110.7650  443.1851
 411.3295   17.5821
  87.3276  674.4307
-147.1598  833.2309
-590.9827 -494.1040
 844.3761 -318.2782
-404.2528 -654.8808
 613.9590 -121.9251
-585.7407 -958.3483
-464.5699  771.0288
[torch.DoubleTensor of size 11x2]



In [23]:
function tsne(vecs, perplexity)
    local opts = {dim = 2, perplexity = perplexity}
    local p = m.embedding.tsne(vecs:double(), opts)
    return p
end